In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
year = 2018
pixel_count = pd.read_csv(os.path.join(os.getcwd(), 'rasters_'+str(year)+'_pixel_counts.csv'))
pixel_count

,n_nonice_2018,n_ice_2018,n_ground_2018,n_water_2018,raster
0,35863561,5897584,113626707,59950901,LS_merged_crs26910_S_2018
1,6730191,899027,24887840,16301218,LS_merged_crs26910_W_2018
2,79013822,3880728,73808926,66038443,LS_merged_crs26911_2018


In [3]:
# total number of pixels in neach category
n_pix = [sum(pixel_count.iloc[:,c]) for c in range(0,4)]
n_pix

[121607574, 10677339, 212323473, 142290562]

In [4]:
# Recreation of sampling design by SEPAL

# ---------------------------------------------
# --------------- PARAMETERS ------------------
# standard error for all the points
std_error = 0.015

# user's accuracies TP/(TP+FP) (estimates)
U = [0.7, 0.6, 0.9, 0.95]
# ---------------------------------------------

# fraction of pixels with a given class in total pixels
total_pix = sum(n_pix)
pix_prop = [n/total_pix for n in n_pix]

# standard deviation of user's accuracies
stdv = [ np.sqrt(u*(1-u)) for u in U]

X = [ x*y for x,y in zip(pix_prop, stdv)]

sample_size = (sum(X)/std_error)**2
sample_size

# distributing sample size among classes
#[...]

454.2894327376922

In [5]:
errors_U =[]
strat_samples = []
strat_title = []

def strat_stderror(U, strat_sample):
    return [ np.sqrt(u*(1-u)/(n-1)) for u,n in zip(U, strat_sample) ]

In [6]:
# Distributing sample size among classes

# ---------------------------------------------
strat_title.append('equal')
sample_equal = [sample_size/4 for i in range(0,4)]
strat_samples.append(sample_equal)
errors_U.append(strat_stderror(U, sample_equal))

# ---------------------------------------------
class2_prop = n_pix[2]/(n_pix[2]+n_pix[3])
class3_prop = n_pix[3]/(n_pix[2]+n_pix[3])

# ---------------------------------------------
strat_title.append('120')
sample_120 = [120, 120, (sample_size-240)*class2_prop, (sample_size-240)*class3_prop] 
strat_samples.append(sample_120)
errors_U.append(strat_stderror(U, sample_120))

# ---------------------------------------------
strat_title.append('100')
sample_100 = [100, 100, (sample_size-200)*class2_prop, (sample_size-200)*class3_prop] 
strat_samples.append(sample_100)
errors_U.append(strat_stderror(U, sample_100))

# ---------------------------------------------
strat_title.append('87')
sample_87 = [87, 87, (sample_size-174)*class2_prop, (sample_size-174)*class3_prop] 
strat_samples.append(sample_87)
errors_U.append(strat_stderror(U, sample_87))

# ---------------------------------------------
strat_title.append('75')
sample_75 = [75, 75, (sample_size-150)*class2_prop, (sample_size-150)*class3_prop] 
strat_samples.append(sample_75)
errors_U.append(strat_stderror(U, sample_75))

# ---------------------------------------------
strat_title.append('prop')
sample_prop = [sample_size*x for x in pix_prop]
strat_samples.append(sample_prop)
errors_U.append(strat_stderror(U, sample_prop))


In [7]:
strat_df = pd.DataFrame(strat_samples).T
strat_df.columns  = strat_title
strat_df

,equal,120,100,87,75,prop
0,113.572358,120.000000,100.000000,87.000000,75.000000,113.463042
1,113.572358,120.000000,100.000000,87.000000,75.000000,9.962236
2,113.572358,128.304782,152.254593,167.821970,182.191856,198.103345
3,113.572358,85.984650,102.034840,112.467463,122.097577,132.760810


In [8]:
errors_df = pd.DataFrame(errors_U).T
errors_df.columns = strat_title
errors_df

,equal,120,100,87,75,prop
0,0.043191,0.042008,0.046057,0.049415,0.053271,0.043212
1,0.046173,0.044909,0.049237,0.052827,0.056949,0.163643
2,0.028275,0.026589,0.024393,0.023227,0.022287,0.021369
3,0.020541,0.023642,0.021683,0.020643,0.019805,0.018987


In [9]:
196*errors_df

,equal,120,100,87,75,prop
0,8.465446,8.233647,9.027097,9.685376,10.441187,8.469559
1,9.049942,8.802139,9.650373,10.354103,11.162098,32.074028
2,5.541935,5.211407,4.781047,4.552508,4.368255,4.188228
3,4.026123,4.633750,4.249788,4.046027,3.881818,3.721429


In [25]:
100 + 100 + 150 + 115
# GOING FOR THE 100 PER MINORITY CLASS

465

In [ ]:
100 + 100 + 150 + 100